In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

from keras import models
from keras import layers
from keras.optimizers import RMSprop
from keras import regularizers
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import metrics
import tensorflow.keras.backend as K

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import *

import os

In [2]:
data = pd.read_csv("./custom_data/train_feature.csv")
X_test = pd.read_csv("./custom_data/test_feature.csv")
sample_submission = pd.read_csv('./data/sample_submission.csv')

In [3]:
X = data.drop('problem', axis=1)
y = data['problem']

In [4]:
indices = np.array(range(X.shape[0]))
num_training_indices = int(0.8 * X.shape[0])
np.random.shuffle(indices)
train_indices = indices[:num_training_indices]
test_indices = indices[num_training_indices:]

X_train, X_val = X.iloc[train_indices], X.iloc[test_indices]
y_train, y_val = y.iloc[train_indices], y.iloc[test_indices]

In [5]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

In [6]:
MODEL_SAVE_FOLDER_PATH = './dnn_model/' 
if not os.path.exists(MODEL_SAVE_FOLDER_PATH): 
    os.mkdir(MODEL_SAVE_FOLDER_PATH)
model_path = MODEL_SAVE_FOLDER_PATH + '{epoch:02d}-{val_loss:.4f}.hdf5'
cb_checkpoint = ModelCheckpoint(filepath=model_path, monitor='val_loss', verbose=1, save_best_only=True)

In [7]:
def mish(x):
    return x * K.tanh(K.softplus(x))

In [8]:
max_features = X_train.shape[1]

model3 = models.Sequential()
model3.add(layers.Dense(512, input_shape=(max_features,), kernel_regularizer=regularizers.l2(0.01)))
model3.add(layers.Activation(mish))
# model3.add(layers.BatchNormalization())
model3.add(layers.Dropout(0.5))
model3.add(layers.Dense(256))
model3.add(layers.Activation(mish))
# model3.add(layers.BatchNormalization())
model3.add(layers.Dropout(0.5))
model3.add(layers.Dense(128))
model3.add(layers.Activation(mish))

model3.add(layers.Dense(64))
model3.add(layers.Activation(mish))
# model3.add(layers.BatchNormalization())
model3.add(layers.Dropout(0.5))
model3.add(layers.Dense(32))
model3.add(layers.Activation(mish))
# model3.add(layers.BatchNormalization())
model3.add(layers.Dropout(0.5))

model3.add(layers.Dense(16))
model3.add(layers.Activation(mish))
# model3.add(layers.BatchNormalization())
model3.add(layers.Dropout(0.5))
model3.add(layers.Dense(4))
model3.add(layers.Activation(mish))
# model3.add(layers.BatchNormalization())
model3.add(layers.Dropout(0.5))
model3.add(layers.Dense(1, activation='sigmoid'))
model3.summary()

# batch_size도 성능에 영향을 미친다. 
model3.compile(optimizer=RMSprop(lr=1e-4), loss='binary_crossentropy', metrics=metrics.AUC(),)
history = model3.fit(X_train, y_train, epochs=500, batch_size=256, 
                    validation_split=0.2, callbacks=[EarlyStopping(patience=5),cb_checkpoint])


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               576512    
_________________________________________________________________
activation (Activation)      (None, 512)               0         
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
activation_1 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               3

In [9]:
pred = model3.predict(X_val)[:,0]

In [10]:
roc_auc_score(y_val,pred)

0.827724149663459

In [11]:
pred = model3.predict(X_test)

In [16]:
a = []
for i in pred:
    a.append(round(i[0]))
pd.DataFrame(a)[0].value_counts()

0    8891
1    6108
Name: 0, dtype: int64

-----

------

In [17]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [18]:
max_features = X.shape[1]

model3 = models.Sequential()
model3.add(layers.Dense(512, input_shape=(max_features,), kernel_regularizer=regularizers.l2(0.01)))
model3.add(layers.Activation(mish))
# model3.add(layers.BatchNormalization())
model3.add(layers.Dropout(0.5))
model3.add(layers.Dense(256))
model3.add(layers.Activation(mish))
# model3.add(layers.BatchNormalization())
model3.add(layers.Dropout(0.5))
model3.add(layers.Dense(128))
model3.add(layers.Activation(mish))

model3.add(layers.Dense(64))
model3.add(layers.Activation(mish))
# model3.add(layers.BatchNormalization())
model3.add(layers.Dropout(0.5))
model3.add(layers.Dense(32))
model3.add(layers.Activation(mish))
# model3.add(layers.BatchNormalization())
model3.add(layers.Dropout(0.5))

model3.add(layers.Dense(16))
model3.add(layers.Activation(mish))
# model3.add(layers.BatchNormalization())
model3.add(layers.Dropout(0.5))
model3.add(layers.Dense(4))
model3.add(layers.Activation(mish))
# model3.add(layers.BatchNormalization())
model3.add(layers.Dropout(0.5))
model3.add(layers.Dense(1, activation='sigmoid'))
model3.summary()

# batch_size도 성능에 영향을 미친다. 
model3.compile(optimizer=RMSprop(lr=1e-4), loss='binary_crossentropy', metrics=metrics.AUC(),)
history2 = model3.fit(X, y, epochs=500, batch_size=256, validation_split=0.2, callbacks=[EarlyStopping(patience=10),cb_checkpoint])


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 512)               576512    
_________________________________________________________________
activation_7 (Activation)    (None, 512)               0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 256)               131328    
_________________________________________________________________
activation_8 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 128)              

In [20]:
pred = model3.predict(X_test)
a = []
for i in pred:
    a.append(round(i[0]))
pd.DataFrame(a)[0].value_counts()

1    11146
0     3853
Name: 0, dtype: int64

In [24]:
model3.save('deep.h5')